# Horovod

HorovodRunner is a general API to run distributed DL workloads on Databricks using Uber’s [Horovod](https://github.com/uber/horovod) framework. By integrating Horovod with Spark’s barrier mode, Databricks is able to provide higher stability for long-running deep learning training jobs on Spark. HorovodRunner takes a Python method that contains DL training code with Horovod hooks. This method gets pickled on the driver and sent to Spark workers. A Horovod MPI job is embedded as a Spark job using barrier execution mode. The first executor collects the IP addresses of all task executors using BarrierTaskContext and triggers a Horovod job using mpirun. Each Python MPI process loads the pickled program back, deserializes it, and runs it.

<br>

![](https://files.training.databricks.com/images/horovod-runner.png)

For additional resources, see:
* [Horovod Runner Docs](https://docs.microsoft.com/en-us/azure/databricks/applications/deep-learning/distributed-training/horovod-runner)
* [Horovod Runner webinar](https://vimeo.com/316872704/e79235f62c)

Run the following cell to set up our environment.

In [0]:
%run "./Includes/Classroom-Setup"

## Build Model

In [0]:
import numpy as np
np.random.seed(0)
import tensorflow as tf
# tf.set_random_seed(42) # For reproducibility
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

def build_model():
  return Sequential([Dense(20, input_dim=8, activation='relu'),
                    Dense(20, activation='relu'),
                    Dense(1, activation='linear')]) # Keep the output layer as linear because this is a regression problem

## Shard Data

From the [Horovod docs](https://github.com/horovod/horovod/blob/master/docs/concepts.rst):

Horovod core principles are based on the MPI concepts size, rank, local rank, allreduce, allgather, and broadcast. These are best explained by example. Say we launched a training script on 4 servers, each having 4 GPUs. If we launched one copy of the script per GPU:

* Size would be the number of processes, in this case, 16.

* Rank would be the unique process ID from 0 to 15 (size - 1).

* Local rank would be the unique process ID within the server from 0 to 3.

We need to shard our data across our processes.  **NOTE:** We are using a Pandas DataFrame for demo purposes. In the next notebook we will use Parquet files with Petastorm for better scalability.

In [0]:
from sklearn.datasets.california_housing import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

def get_dataset(rank=0, size=1):
  scaler = StandardScaler()
  cal_housing = fetch_california_housing(data_home="/dbfs/ml/" + str(rank) + "/")
  X_train, X_test, y_train, y_test = train_test_split(cal_housing.data,
                                                       cal_housing.target,
                                                       test_size=0.2,
                                                       random_state=1)
  scaler.fit(X_train)
  X_train = scaler.transform(X_train[rank::size])
  y_train = y_train[rank::size]
  X_test = scaler.transform(X_test[rank::size])
  y_test = y_test[rank::size]
  return (X_train, y_train), (X_test, y_test)

## Horovod

In [0]:
from tensorflow.keras import optimizers
import horovod.tensorflow.keras as hvd
from tensorflow.keras import backend as K

def run_training_horovod():
  # Horovod: initialize Horovod.
  hvd.init()
  # If using GPU: pin GPU to be used to process local rank (one GPU per process)
  # config = tf.ConfigProto()
  # config.gpu_options.allow_growth = True
  # config.gpu_options.visible_device_list = str(hvd.local_rank())
  # K.set_session(tf.Session(config=config))
  print(f"Rank is: {hvd.rank()}")
  print(f"Size is: {hvd.size()}")
  
  (X_train, y_train), (X_test, y_test) = get_dataset(hvd.rank(), hvd.size())
  
  model = build_model()
  
  from tensorflow.keras import optimizers
  # Horovod: adjust learning rate based on number of GPUs/CPUs.
  optimizer = optimizers.Adam(lr=0.001*hvd.size())
  
  # Horovod: add Horovod Distributed Optimizer.
  optimizer = hvd.DistributedOptimizer(optimizer)

  model.compile(optimizer=optimizer, loss="mse", metrics=["mse"])
  
  history = model.fit(X_train, y_train, validation_split=.2, epochs=10, batch_size=64, verbose=2)

Test it out on just the driver.

In [0]:
from sparkdl import HorovodRunner
hr = HorovodRunner(np=-1)
hr.run(run_training_horovod)

HorovodRunner will stream all training logs to notebook cell output. If there are too many logs, you
can adjust the log level in your train method. Or you can set driver_log_verbosity to
'log_callback_only' and use a HorovodRunner log callback on the first worker to get concise
progress updates.
HorovodRunner will launch Horovod jobs on the driver node. There would be resource contention if you
share the cluster with others.
The global names read or written to by the pickled function are {'print', 'hvd', 'get_dataset', 'build_model'}.
The pickled object size is 2831 bytes.

### How to enable Horovod Timeline? ###
HorovodRunner has the ability to record the timeline of its activity with Horovod Timeline. To
record a Horovod Timeline, set the `HOROVOD_TIMELINE` environment variable to the location of the
timeline file to be created. You can then open the timeline file using the chrome://tracing
facility of the Chrome browser.

Executing command: ['mpirun', '--allow-run-as-root', '-np', '1', '-H', 'localhost', '--stdin', 'none', '--tag-output', '-mca', 'rmaps', 'seq', '--bind-to', 'none', '-x', 'NCCL_DEBUG=INFO', '-mca', 'pml', 'ob1', '-mca', 'btl', '^openib', '-mca', 'plm_rsh_agent', 'ssh -o StrictHostKeyChecking=no -i /tmp/HorovodRunner_5d75ed17755a/id_rsa', 'bash', '/tmp/HorovodRunner_5d75ed17755a/launch.sh'].

[1,0]<stdout>:Rank is: 0
[1,0]<stdout>:Size is: 1
[1,0]<stderr>:Downloading Cal. housing from https://ndownloader.figshare.com/files/5976036 to /dbfs/ml/0/
[1,0]<stderr>:Downloading Cal. housing from https://ndownloader.figshare.com/files/5976036 to /dbfs/ml/0/
[1,0]<stderr>:2022-03-09 09:36:18.184488: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN)to use the following CPU instructions in performance-critical operations: AVX2 AVX512F FMA
[1,0]<stderr>:To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
[1,0]<stderr>:2022-03-09 09:36:18.202582: I tensorflow/core/platform/profile_utils/cpu_utils.cc:104] CPU Frequency: 2095074999 Hz
[1,0]<stderr>:2022-03-09 09:36:18.203033: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x564cab93dfa0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
[1,0]<stderr>:2022-03-09 09:36:18.203086: I tensorflow/compiler/xla/service/service.cc:176] StreamExecutor device (0): Host, Default Version
[1,0]<stdout>:Epoch 1/10
[1,0]<stdout>:207/207 - 0s - loss: 3.4254 - mse: 3.4254 - val_loss: 1.0853 - val_mse: 1.0853
[1,0]<stdout>:Epoch 2/10
[1,0]<stdout>:207/207 - 0s - loss: 0.8421 - mse: 0.8421 - val_loss: 0.7221 - val_mse: 0.7221
[1,0]<stdout>:Epoch 3/10
[1,0]<stdout>:207/207 - 0s - loss: 0.6505 - mse: 0.6505 - val_loss: 0.5928 - val_mse: 0.5928
[1,0]<stdout>:Epoch 4/10
[1,0]<stdout>:207/207 - 0s - loss: 0.5393 - mse: 0.5393 - val_loss: 0.5145 - val_mse: 0.5145
[1,0]<stdout>:Epoch 5/10
[1,0]<stdout>:207/207 - 0s - loss: 0.4727 - mse: 0.4727 - val_loss: 0.4708 - val_mse: 0.4708
[1,0]<stdout>:Epoch 6/10
[1,0]<stdout>:207/207 - 0s - loss: 0.4391 - mse: 0.4391 - val_loss: 0.4530 - val_mse: 0.4530
[1,0]<stdout>:Epoch 7/10
[1,0]<stdout>:207/207 - 0s - loss: 0.4171 - mse: 0.4171 - val_loss: 0.4228 - val_mse: 0.4228
[1,0]<stdout>:Epoch 8/10
[1,0]<stdout>:207/207 - 0s - loss: 0.3990 - mse: 0.3990 - val_loss: 0.4111 - val_mse: 0.4111
[1,0]<stdout>:Epoch 9/10
[1,0]<stdout>:207/207 - 0s - loss: 0.3844 - mse: 0.3844 - val_loss: 0.3953 - val_mse: 0.3953
[1,0]<stdout>:Epoch 10/10
[1,0]<stdout>:207/207 - 0s - loss: 0.3798 - mse: 0.3798 - val_loss: 0.3875 - val_mse: 0.3875

# Better Horovod

In [0]:
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import *

def run_training_horovod():
  # Horovod: initialize Horovod.
  hvd.init()
  # If using GPU: pin GPU to be used to process local rank (one GPU per process)
  # config = tf.ConfigProto()
  # config.gpu_options.allow_growth = True
  # config.gpu_options.visible_device_list = str(hvd.local_rank())
  # K.set_session(tf.Session(config=config))
  
  
  
  print(f"Rank is: {hvd.rank()}")
  print(f"Size is: {hvd.size()}")
  
  (X_train, y_train), (X_test, y_test) = get_dataset(hvd.rank(), hvd.size())
  
  model = build_model()
  
  from tensorflow.keras import optimizers
  # Horovod: adjust learning rate based on number of GPUs.
  optimizer = optimizers.Adam(lr=0.001*hvd.size())
  
  # Horovod: add Horovod Distributed Optimizer.
  optimizer = hvd.DistributedOptimizer(optimizer)

  model.compile(optimizer=optimizer, loss="mse", metrics=["mse"])
  ml_working_path = "pinky.gtm@mail.kmutt.ac.th"
  # Use the optimized FUSE Mount
  checkpoint_dir = f"{ml_working_path}/horovod_checkpoint_weights.ckpt"
  
  callbacks = [
    # Horovod: broadcast initial variable states from rank 0 to all other processes.
    # This is necessary to ensure consistent initialization of all workers when
    # training is started with random weights or restored from a checkpoint.
    hvd.callbacks.BroadcastGlobalVariablesCallback(0),

    # Horovod: average metrics among workers at the end of every epoch.
    # Note: This callback must be in the list before the ReduceLROnPlateau,
    # TensorBoard or other metrics-based callbacks.
    hvd.callbacks.MetricAverageCallback(),

    # Horovod: using `lr = 1.0 * hvd.size()` from the very beginning leads to worse final
    # accuracy. Scale the learning rate `lr = 1.0` ---> `lr = 1.0 * hvd.size()` during
    # the first five epochs. See https://arxiv.org/abs/1706.02677 for details.
    hvd.callbacks.LearningRateWarmupCallback(warmup_epochs=5, verbose=1),
    
    # Reduce the learning rate if training plateaus.
    ReduceLROnPlateau(patience=10, verbose=1)
  ]
  
  # Horovod: save checkpoints only on worker 0 to prevent other workers from corrupting them.
  if hvd.rank() == 0:
    callbacks.append(ModelCheckpoint(checkpoint_dir, save_weights_only=True))
  
  history = model.fit(X_train, y_train, validation_split=.2, epochs=10, batch_size=64, verbose=2, callbacks=callbacks)

Test it out on just the driver.

In [0]:
from sparkdl import HorovodRunner
hr = HorovodRunner(np=-1)
hr.run(run_training_horovod)

HorovodRunner will stream all training logs to notebook cell output. If there are too many logs, you
can adjust the log level in your train method. Or you can set driver_log_verbosity to
'log_callback_only' and use a HorovodRunner log callback on the first worker to get concise
progress updates.
HorovodRunner will launch Horovod jobs on the driver node. There would be resource contention if you
share the cluster with others.
The global names read or written to by the pickled function are {'ModelCheckpoint', 'hvd', 'build_model', 'ReduceLROnPlateau', 'print', 'get_dataset'}.
The pickled object size is 3348 bytes.

### How to enable Horovod Timeline? ###
HorovodRunner has the ability to record the timeline of its activity with Horovod Timeline. To
record a Horovod Timeline, set the `HOROVOD_TIMELINE` environment variable to the location of the
timeline file to be created. You can then open the timeline file using the chrome://tracing
facility of the Chrome browser.

Executing command: ['mpirun', '--allow-run-as-root', '-np', '1', '-H', 'localhost', '--stdin', 'none', '--tag-output', '-mca', 'rmaps', 'seq', '--bind-to', 'none', '-x', 'NCCL_DEBUG=INFO', '-mca', 'pml', 'ob1', '-mca', 'btl', '^openib', '-mca', 'plm_rsh_agent', 'ssh -o StrictHostKeyChecking=no -i /tmp/HorovodRunner_bbe999b468f4/id_rsa', 'bash', '/tmp/HorovodRunner_bbe999b468f4/launch.sh'].

[1,0]<stdout>:Rank is: 0
[1,0]<stdout>:Size is: 1
[1,0]<stderr>:2022-03-09 09:40:53.167713: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN)to use the following CPU instructions in performance-critical operations: AVX2 AVX512F FMA
[1,0]<stderr>:To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
[1,0]<stderr>:2022-03-09 09:40:53.173692: I tensorflow/core/platform/profile_utils/cpu_utils.cc:104] CPU Frequency: 2095074999 Hz
[1,0]<stderr>:2022-03-09 09:40:53.174133: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x5557872a3310 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
[1,0]<stderr>:2022-03-09 09:40:53.174183: I tensorflow/compiler/xla/service/service.cc:176] StreamExecutor device (0): Host, Default Version
[1,0]<stdout>:Epoch 1/10
[1,0]<stdout>:207/207 - 0s - loss: 1.4516 - mse: 1.4516 - val_loss: 0.6863 - val_mse: 0.6863
[1,0]<stdout>:Epoch 2/10
[1,0]<stdout>:207/207 - 0s - loss: 0.6057 - mse: 0.6057 - val_loss: 0.5098 - val_mse: 0.5098
[1,0]<stdout>:Epoch 3/10
[1,0]<stdout>:207/207 - 0s - loss: 0.4758 - mse: 0.4758 - val_loss: 0.4532 - val_mse: 0.4532
[1,0]<stdout>:Epoch 4/10
[1,0]<stdout>:207/207 - 0s - loss: 0.4273 - mse: 0.4273 - val_loss: 0.4240 - val_mse: 0.4240
[1,0]<stdout>:Epoch 5/10
[1,0]<stdout>:
[1,0]<stdout>:Epoch 5: finished gradual learning rate warmup to 0.001.
[1,0]<stdout>:207/207 - 0s - loss: 0.4077 - mse: 0.4077 - val_loss: 0.4181 - val_mse: 0.4181
[1,0]<stdout>:Epoch 6/10
[1,0]<stdout>:207/207 - 0s - loss: 0.3955 - mse: 0.3955 - val_loss: 0.4034 - val_mse: 0.4034
[1,0]<stdout>:Epoch 7/10
[1,0]<stdout>:207/207 - 0s - loss: 0.3874 - mse: 0.3874 - val_loss: 0.3987 - val_mse: 0.3987
[1,0]<stdout>:Epoch 8/10
[1,0]<stdout>:207/207 - 0s - loss: 0.3806 - mse: 0.3806 - val_loss: 0.3942 - val_mse: 0.3942
[1,0]<stdout>:Epoch 9/10
[1,0]<stdout>:207/207 - 0s - loss: 0.3748 - mse: 0.3748 - val_loss: 0.3882 - val_mse: 0.3882
[1,0]<stdout>:Epoch 10/10
[1,0]<stdout>:207/207 - 0s - loss: 0.3694 - mse: 0.3694 - val_loss: 0.3836 - val_mse: 0.3836

## Run on all workers

In [0]:
from sparkdl import HorovodRunner
hr = HorovodRunner(np=0)
hr.run(run_training_horovod)

HorovodRunner will stream all training logs to notebook cell output. If there are too many logs, you
can adjust the log level in your train method. Or you can set driver_log_verbosity to
'log_callback_only' and use a HorovodRunner log callback on the first worker to get concise
progress updates.
Setting np=0 is deprecated and it will be removed in the next major Databricks Runtime release.
Choosing np based on the total task slots at runtime is unreliable due to dynamic executor
registration. Please set the number of parallel processes you need explicitly.
The global names read or written to by the pickled function are {'ModelCheckpoint', 'hvd', 'build_model', 'ReduceLROnPlateau', 'print', 'get_dataset'}.
The pickled object size is 3350 bytes.

### How to enable Horovod Timeline? ###
HorovodRunner has the ability to record the timeline of its activity with Horovod Timeline. To
record a Horovod Timeline, set the `HOROVOD_TIMELINE` environment variable to the location of the
timeline file to be created. You can then open the timeline file using the chrome://tracing
facility of the Chrome browser.

WARNING:root:Waiting for executor registration before HorovodRunner could start. 0 seconds / 120 seconds
WARNING:root:Waiting for executor registration before HorovodRunner could start. 5 seconds / 120 seconds
WARNING:root:Waiting for executor registration before HorovodRunner could start. 10 seconds / 120 seconds
WARNING:root:Waiting for executor registration before HorovodRunner could start. 15 seconds / 120 seconds
WARNING:root:Waiting for executor registration before HorovodRunner could start. 20 seconds / 120 seconds
WARNING:root:Waiting for executor registration before HorovodRunner could start. 25 seconds / 120 seconds